<h1 style="text-align:center;font-size:48pt;color:#f1faee; text-shadow: 0px 0px 8px #101010;">Walmart Sales Forecasting</h1>

<img src="https://media.giphy.com/media/3o85xBiolEFwFtOw3C/giphy.gif" width="800" height="300">

In [ ]:
#Import Important libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import plotly.express as px
from fbprophet.plot import plot_plotly, plot_components_plotly
from fbprophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose as sd
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import missingno as msno
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load our data

features = pd.read_csv("../input/walmart-sales-prediction/features.csv", parse_dates=['Date'])
stores = pd.read_csv("../input/walmart-sales-prediction/stores.csv")
train = pd.read_csv("../input/walmart-sales-prediction/train.csv", parse_dates=['Date'])
test = pd.read_csv("../input/walmart-sales-prediction/test.csv", parse_dates=['Date'])

In [ ]:
# Display the first 3 rows

print(features.head(3))
print('\n')
print(stores.head(3))
print('\n')
print(train.head(3))

In [ ]:
# Display the dataset shape

print(features.shape)
print(stores.shape) 
print(train.shape)
print(train)

In [ ]:
from xgboost import XGBRegressor 

xgb = XGBRegresssor()
xgb.fit(X_train_standard, y_train)

plotLMResults(xgb, X_train_standard, X_test_standard)

# stores

In [ ]:
# Stores Time series
sts = train.groupby(["Store","Date"])["Weekly_Sales"].sum().reset_index()

In [ ]:
# Display top rows
sts.head()

In [ ]:
vsts = sts.groupby(["Store"])["Weekly_Sales"].agg(["sum","mean"]).reset_index()
fig = px.bar(vsts, x='Store', y='sum',
             hover_data=['Store', 'sum', 'mean'], color='mean',
             labels={'sum':'Weekly Sales'}, height=400)
fig.show()

In [ ]:
# Walmart weekly sales by Store and Date(Using Plotly)
fig = go.Figure()
for s in sts.Store.unique():
    fig.add_trace(
        go.Scatter(
            x=sts[sts.Store==s].Date,
            y=sts[sts.Store==s].Weekly_Sales,
            name="Store_"+str(s)
        ))
fig.show()

The highest sales were on Dec/24 and Dec/23, these are thanksgiving holidays.

In [ ]:
l_sts = pd.DataFrame()
for s in sts.Store.unique():
    df = pd.DataFrame(sts[sts.Store==s])
    for i in range(26, 54):
        df[f"l{i}"] = df.Weekly_Sales.shift(i)
    df.dropna(inplace=True)    
    l_sts = l_sts.append(df)

In [ ]:
l_sts = l_sts.set_index("Date")
l_sts.head()

In [ ]:
def sts_train_test_split(l_sts, test_size):
    train_set = pd.concat([l_sts[l_sts.Store==s].iloc[:-int(test_size*len(l_sts[l_sts.Store==1]))+1] for s in l_sts.Store.unique()])
    test_set = pd.concat([l_sts[l_sts.Store==s].iloc[-int(test_size*len(l_sts[l_sts.Store==1]))+1:] for s in l_sts.Store.unique()])
    
    y_train = train_set.Weekly_Sales
    X_train = train_set.drop(['Weekly_Sales'], axis=1)
    
    y_test = test_set.Weekly_Sales
    X_test = test_set.drop(['Weekly_Sales'], axis=1)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = sts_train_test_split(l_sts, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
standard_scaler = StandardScaler()
X_train_standard = pd.DataFrame(standard_scaler.fit_transform(X_train)).set_index(X_train.index)
X_test_standard =  pd.DataFrame(standard_scaler.transform(X_test)).set_index(X_test.index)

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train_standard, y_train)

In [ ]:
pred = xgb.predict(X_test_standard)
matest = X_test.copy()
matest["Pred_Sales"] = pred
matest["Actual_Sales"] = y_test
matest.head()

In [ ]:
fig = go.Figure()
for s in range(1,5):
    fig.add_trace(
        go.Scatter(
            x=matest[matest.Store==s].index,
            y=matest[matest.Store==s].Pred_Sales,
            name="Store_"+str(s)+"_pred"
        ))
    fig.add_trace(
        go.Scatter(
            x=matest[matest.Store==s].index,
            y=matest[matest.Store==s].Actual_Sales,
            name="Store_"+str(s)+"_actual",
            line = dict(shape = 'linear', color = 'rgb(255, 12, 24)', width=0.7, dash = 'dash')
        ))
error = mean_absolute_percentage_error(pred, y_test)
fig.update_layout(
    title=f"Mean Absolute Percentage Error: {error:.2f}%",
    xaxis_title="weeks",
    yaxis_title="sales",
    yaxis_tickprefix = '$',
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ))
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
store = tdf.groupby(["Store","Size","Type"])["Weekly_Sales"].sum().reset_index()


fig = px.bar(store, x='Store', y="Weekly_Sales",
             hover_data=['Store', 'Weekly_Sales'], color='Type',height=400, title="Weekly_Sales by Store Type")
fig.show()


fig = px.bar(store, x='Store', y="Weekly_Sales",
             hover_data=['Store', 'Size'], color='Size', height=400, title="Weekly_Sales by Store Size")
fig.show()

In [ ]:
lxgb_sts = pd.DataFrame()
for s in sts.Store.unique():
    df = pd.DataFrame(sts[sts.Store==s]).set_index("Date")
    for i in range(26, 54):
        df[f"l{i}"] = df.Weekly_Sales.shift(i)
    
    df.dropna(inplace=True)    
    lxgb_sts = lxgb_sts.append(df)
    
for i in range(0,12):
        lxgb_sts[f"h{i}"] = lxgb_sts.index.map(lambda x: holidays[x-timedelta(weeks=i)])
        lxgb_sts[f"t{i}"] = lxgb_sts.index.map(lambda x: temperature[x-timedelta(weeks=i)])
        lxgb_sts[f"f{i}"] = lxgb_sts.index.map(lambda x: fuel_price[x-timedelta(weeks=i)])
        
lxgb_sts["Size"] = lxgb_sts.Store.map(lambda x: store[store.Store==x]["Size"].item())
lxgb_sts["Type"] = lxgb_sts.Store.map(lambda x: store[store.Store==x]["Type"].item()).astype('category').cat.codes

In [ ]:
X_train, X_test, y_train, y_test = sts_train_test_split(lxgb_sts, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
standard_scaler = StandardScaler()
X_train_standard = pd.DataFrame(standard_scaler.fit_transform(X_train)).set_index(X_train.index)
X_test_standard =  pd.DataFrame(standard_scaler.transform(X_test)).set_index(X_test.index)

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train_standard, y_train)

In [ ]:
pred = xgb.predict(X_test_standard)
matest = X_test.copy()
matest["Pred_Sales"] = pred
matest["Actual_Sales"] = y_test
matest.head()

In [ ]:
fig = go.Figure()
for s in range(1,5):
    fig.add_trace(
        go.Scatter(
            x=matest[matest.Store==s].index,
            y=matest[matest.Store==s].Pred_Sales,
            name="Store_"+str(s)+"_pred"
        ))
    fig.add_trace(
        go.Scatter(
            x=matest[matest.Store==s].index,
            y=matest[matest.Store==s].Actual_Sales,
            name="Store_"+str(s)+"_actual",
            line = dict(shape = 'linear', color = 'rgb(255, 12, 24)', width=0.7, dash = 'dash')
        ))
error = mean_absolute_percentage_error(pred, y_test)
fig.update_layout(
    title=f"Mean Absolute Percentage Error: {error:.2f}%",
    xaxis_title="weeks",
    yaxis_title="sales",
    yaxis_tickprefix = '$',
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ))
fig.update_xaxes(rangeslider_visible=True)
fig.show()

***